In [ ]:
import numpy as np
import os
import mne
import pickle
import pandas as pd
from mat73 import loadmat
import matplotlib.pyplot as plt
import osl
import yaml

In [ ]:
# Load raw data
raw = loadmat('/well/woolrich/projects/disp_csaky/opm_lukas/data.mat')

In [ ]:
raw = raw['data']

In [ ]:
# Select bad channels
#drop_inds = [78, 79, 80, 30, 31, 32, 48, 49, 50, 63, 64, 65, 12, 13, 14, 3, 4, 5, 0, 1, 2, 6, 7, 8, 105, 106, 107]
#good_inds = [i for i in range(186) if i not in drop_inds]

drop_inds = [12, 13, 14, 42, 43, 44, 57, 58, 59, 81, 82, 83, 111, 112, 113, 159, 160, 161, 165, 166, 167, 174, 175, 176]
good_inds = [i for i in range(189) if i not in drop_inds]

ch_names = [str(i) for i in good_inds]

In [ ]:
# Drop bad channels
raw = raw[good_inds, :]

In [ ]:
data = raw

In [ ]:
# Load channel locations
channels = pd.read_csv('/well/woolrich/projects/disp_csaky/opm_lukas/20221019_085012_channels.tsv', sep='\t')

In [ ]:
# Set channel locations
chn_dict = []
for i in range(len(channels)):
    if i in good_inds:
        chn_positions = np.array([channels['Px'][i], channels['Py'][i], channels['Pz'][i], 0, 0, 0, 0, 0, 0, 0, 0, 0])

        chd = {'loc': chn_positions, 'ch_name': channels['name'][i], 'kind': 'FIFFV_EEG_CH'}
        chn_dict.append(chd)

In [ ]:
# Create info structure
ch_names = [d['ch_name'] for d in chn_dict]
info = mne.create_info(ch_names=ch_names, sfreq=1200, ch_types='eeg')

# set channel locations
for i in range(len(chn_dict)):
    info['chs'][i]['loc'] = chn_dict[i]['loc']

In [ ]:
# create mne raw object
raw = mne.io.RawArray(data, info)

In [ ]:
%matplotlib widget
raw.plot_psd(fmax=100, n_fft=10000)
e=0

In [ ]:
raw.plot_sensors(show_names=True)

In [ ]:
# OSL preproc pipeline
outdir = '/well/woolrich/projects/disp_csaky/opm_lukas/osl_mark/'

config_text = """
meta:
  event_codes:
    words/hungry: 2
    words/tired: 3
    words/thirsty: 4
    words/toilet: 5
    words/pain: 6
preproc:
  - filter:         {l_freq: 1, h_freq: 40, method: 'iir', iir_params: {order: 5, ftype: butter}}
  - bad_channels:   {picks: 'eeg', significance_level: 0.4}        
  - bad_segments:   {segment_len: 200, picks: 'eeg', significance_level: 0.1}
  - bad_segments:   {segment_len: 400, picks: 'eeg', significance_level: 0.1}
  - bad_segments:   {segment_len: 600, picks: 'eeg', significance_level: 0.1}
  - bad_segments:   {segment_len: 800, picks: 'eeg', significance_level: 0.1}
  - ica_raw:        {picks: 'eeg', n_components: 50}
"""

In [ ]:
# save raw data
raw.save(outdir + 'raw.fif', overwrite=True)

In [ ]:
# run OSL preprocessing
config = yaml.load(config_text, Loader=yaml.FullLoader)
dataset = osl.preprocessing.run_proc_chain(config,
                                           outdir + 'raw.fif',
                                           outdir=outdir,
                                           overwrite=True,
                                           gen_report=False)

In [ ]:
# drop bad channels detected by OSL
raw_proc = dataset['raw']
raw_proc = raw_proc.drop_channels(raw_proc.info['bads'])

In [ ]:
# select x, y, z channels
x_inds = [i for i, c in enumerate(raw_proc.ch_names) if '[X]' in c]
y_inds = [i for i, c in enumerate(raw_proc.ch_names) if '[Y]' in c]
z_inds = [i for i, c in enumerate(raw_proc.ch_names) if '[Z]' in c]

In [ ]:
ica = dataset['ica']

In [ ]:
# Get components and sources
comps = ica.get_components()
ica_sources = ica.get_sources(raw_proc).get_data()

In [ ]:
ch_names = [c for c in raw_proc.ch_names if '[Z]' in c]
info_z = raw_proc.copy().pick_channels(ch_names).info

In [ ]:
# plot component timeseries and topographies
%matplotlib inline
index = 26

mne.viz.plot_topomap(comps[z_inds, index], info_z)

%matplotlib widget
plt.plot(ica_sources[index][40000:80000], linewidth=0.5)

In [ ]:
# set components to exclude
#ica.exclude = [6, 12, 15]
#ica.exclude = [3, 4, 10]
ica.exclude = [1, 2, 11]

In [ ]:
# apply ica to the data
raw_proc = ica.apply(raw_proc)

In [ ]:
# save data
raw_proc.save(outdir + 'raw_preproc.fif')